# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Francisco Gallucci  <br>
Url: https://github.com/fjaviergallucci/ao-tp-algoritmo-genetico<br>
Colab:  https://colab.research.google.com/drive/1RcZmnhmov3Dqthi6ICDtzqbWRMvjA8Og?usp=sharing<br>
Colab codigo:  https://colab.research.google.com/drive/1j4jRu21Birtrrf8gxZf5plSmFaFBvHnD?usp=sharing<br>
Problema:
>3. Combinar cifras y operaciones

Descripción del problema:(copiar enunciado)
-   El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.
-   Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
-   Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad
dada. Un ejemplo sería para obtener el 4:
-   Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las siguientes cuestiones:
-   ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?
-   ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo ?
-   Nota: Es posible usar la función de python “eval” para evaluar una expresión:
....

(*) La respuesta es obligatoria

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta
Entendiendo las restricciones por:
-   Conjunto de números del 1 al 9
-   Conjunto de operadores: +-*/
-   No se pueden repetir los números ni operadores
-   Estructura es un número seguido de un operador, por ejemplo "1+2-3*4/5", para un total de 5 números y los 4 operadores, 9 elementos

Teniendo en cuenta las restricciones anteriores, se calculan las permutaciones sin repetición de números agrupados en 5, luego de operadores agrupados en 4 y se multiplican ambos. La cantidad de permutaciones sin repetición para los números es 15120. La cantidad de permutaciones sin repetición para los operadores es 24. Multiplicando luego 15120*24=362880

Sin tener en cuenta las restricciones anteriores, quiere decir que se pueden repetir los elementos, y, no tiene que respetar la estructura, por lo tanto puede tener cualquier longitud. Se calculan las permutaciones con repetición del total de elementos, que seria "123456789+-*/", obteniendo como resultado un total de 259459200 permutaciones

In [1]:
from itertools import permutations

_NUMBERS = '123456789'
_OPERATIONS = '+-*/'

# Calculando sin repeticiones
number_permutations_count = sum(1 for _ in permutations(_NUMBERS, 5))
operators_permutations_count = sum(1 for _ in permutations(_OPERATIONS, 4))
total_permutations = number_permutations_count * operators_permutations_count
print(f'Permutaciones sin repeticion: {total_permutations}')


# Calculando con repeticiones
total_permutations = sum(1 for _ in permutations(_NUMBERS+_OPERATIONS, 9))
print(f'Permutaciones con repeticion: {total_permutations}')

Permutaciones sin repeticion: 362880
Permutaciones con repeticion: 259459200


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta:
Diccionario clave -> valor. Como clave la expresion y como valor la evaluación de esa expresión

De esa forma el guardar y buscar expresiones es mucho más rápido ya que solo se accede al índice en el diccionario directamente. Y en el caso de búsqueda por valor, como ya está calculada la eval, solo se accede a la misma a través de la clave en el diccionario

In [2]:
expression1 = "1+2-3*4/5"
expression2 = "3+4/5*7-8"
population = { expression1 : eval(expression1), expression2 : eval(expression2) }
print(population)

{'1+2-3*4/5': 0.6000000000000001, '3+4/5*7-8': 0.6000000000000014}


Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta pregunta función objetivo
En el caso de la búsqueda de un número, la función objetivo es que la expresión generada de como resultado el número buscado.
En el caso de conseguir el máximo, mínimo y números enteros intermedios de todas las permutaciones, es solo asegurarse se generan todas las posibles

Respuesta pregunta maximización o minimización
Ninguno, es de búsqueda

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [4]:
from itertools import permutations
from itertools import chain
from itertools import zip_longest
import time
_NUMBERS = '123456789'
_OPERATIONS = '+-*/'
_EXPRESSION_PATTERN = r'^[1-9]{1}[\-\+\*\/]{1}[1-9]{1}[\-\+\*\/]{1}[1-9]{1}[\-\+\*\/]{1}[1-9]{1}[\-\+\*\/]{1}[1-9]$'

def find_expressions(target: int = None):
    """Funcion principal que genera todas las permutaciones o expresiones posibles y devuelve:

    Args:
      target: El numero para el cual se desan generar expresiones. Si no se pasa el target, se generaran todas las posibles permutaciones. Si se define el target, se devolveran solo las permutaciones que al ser evaluadas den ese numero

    Returns:
      number_permutations: Todas las permutaciones encontradas
      integer_permutations: Permutaciones cuya evaluaciones dan numeros enteros
      max: Permutacion que devuelve valor maximo
      min: Permutacion que devuelve valor minimo
    """
    number_permutations = {}
    integer_permutations = {}
    max = "0"
    min = "0"
    # Como las permutaciones de numeros son cientos de miles, usamos un generador para ahorrar memoria y tiempo de procesamiento
    for number in generate_number_yield():
        # En el caso particulas de las operaciones se puede pasar este generador a un lista. Pero no hace ninguna diferencia en recursos. Mejor seguir usando generadores
        for operation in generate_operations_yield():
            # Como ya tenemos todas las posibles permutaciones de operaciones. Lo que hay que hacer es jutnar el numero generado con el orden de los operadores y asi se obtienen todas las permutacioens de un numero con los operadores
            expression = generate_expresion(number, operation)
            value = eval(expression)

            #Si estamos buscando un target y la expresion generada lo cumple, la devolvemos, sino, continuamos
            if target is not None and value == target:
                return {expression: value}, None, None, None
            elif target is None:
                number_permutations[expression] = value
                # Verificamos si es el nuevo maximo
                if value > eval(max):
                    max = expression
                # Verificamos si es el nuevo minimo
                if value < eval(min):
                    min = expression
                # Lo guardamos si el resultado de la expresion es entero
                if abs(value - int(value)) == 0.0:
                    integer_permutations[expression] = value
    # print(number_combinations)
    return number_permutations, integer_permutations, max, min


def generate_number_yield():
    """Genera todas las posibles permutaciones de numeros

    Returns:
      Yield con el numero o permutacion generada.
    """
    for number in permutations(_NUMBERS, 5):
        yield number


def generate_operations_yield():
    """Esta funcion generara todas las posibles permutaciones de operadores

    Returns:
      Yield con la permutacion generada de los operadores
    """
    for operation in permutations(_OPERATIONS, 4):
        yield operation


def generate_expresion(number: str, operators: str):
    """Genera la expresion dado los numeros y los operadores

    La funcion recorre ambos strings y genera una combinacion de ambos respetando el orden y la estructura esperada

    Args:
      number: Permutacion de los numeros, por ejemplo "37586"
      operators: Permutacion de los operadores, por ejemplo: "+/*-"

    Returns:
      Expression generada
    """
    return "".join((filter(lambda x: x != '', chain.from_iterable(
        zip_longest(number, operators, fillvalue='')))))


In [5]:
#Buscar expresiones para el numero
_NUMBER_TO_SEARCH = 4
start_time = time.time()
expressions, _, _, _ = find_expressions(_NUMBER_TO_SEARCH)
print("--- %s segundos ---" % (time.time() - start_time))
print(f'Expresion encontrada: {expressions}')

--- 0.0021932125091552734 segundos ---
Expresion encontrada: {'1-2*3/6+4': 4.0}


In [6]:
#Encontrar max, min, enteros intermedios
start_time = time.time()
expressions, integer_expressions, max, min = find_expressions()
print("--- %s segundos ---" % (time.time() - start_time))
print(f'Cantidad de expresiones: {len(expressions)}')
if max in expressions:
    print(f'Max = {max} = {expressions[max]}')
if min in expressions:
    print(f'Min = {min} = {expressions[min]}')
print(f'Cantidad de expresiones con valores enteros: {len(integer_expressions)}')
print(f'Primeras 10 expresiones con valores enteros encontradas:')
count = 0
for expresion in expressions:
    if count <= 10:
        count += 1
        print(expresion)

--- 6.920217990875244 segundos ---
Cantidad de expresiones: 362880
Max = 7-1/6+8*9 = 78.83333333333333
Min = 1+2/7-8*9 = -70.71428571428571
Cantidad de expresiones con valores enteros: 90000
Primeras 10 expresiones con valores enteros encontradas:
1+2-3*4/5
1+2-3/4*5
1+2*3-4/5
1+2*3/4-5
1+2/3-4*5
1+2/3*4-5
1-2+3*4/5
1-2+3/4*5
1-2*3+4/5
1-2*3/4+5
1-2/3+4*5


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

In [ ]:
Hay dos sub-problemas; el de encontrar un número; y buscar max, min y enteros intermedios de todas las combinaciones. Se asume que la pregunta es sobre el buscar una expresión para un número dado.

Peor de los casos es O(n^2)

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

De igual forma hay dos sub-problemas, no queda claro para cual se hace la pregunta.

Respuesta
Se diseñó un algoritmo genético que dada una población inicial puede: Buscar una expresión que satisfaga un número dado, y buscar el min, max y enteros intermedios de las posibles permutaciones.

En el caso de buscar una expresión que satisfaga un número dado, en algunos casos se consigue la expresión más rápido que por fuerza bruta, pero también pudiera tardar más. Esto debido a que tiene un componente de aleatoriedad y mientras las generaciones generadas no contengan la solución, seguirá generando.

En el caso de buscar el max, min y enteros intermedios. Se mejora mucho el tiempo para encontrar los mismos valores de max y min, pero, el AG no garantiza que los valores enteros intermedios encontrados sean todos los existentes.

(*)Calcula la complejidad del algoritmo 

Respuesta
Para la función principal del algoritmo genético la complejidad es constante, ya que depende de la cantidad de generaciones que se quieran generar y la cantidad de sujetos para cada población.

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta
